<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/dev/Archivo_de_cierre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerías necesarias

Nota: openpyxl ya está por defecto en colab

In [3198]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils import get_column_letter
import datetime

# Limpieza y transformación de SICAR Ventas

In [3199]:
ventas = pd.read_excel('Ventas.xlsx', sheet_name=0, header=5)
ventas.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Documento,Unnamed: 1,Unnamed: 2,Fecha,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Total,Unnamed: 31,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,NaN,NaN,2025-07-03 08:03:19,NaN,63015.0,NaN,Público en General,NaN,NaN,...,$ 49.00,NaN,$ 49.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
1,PZA,1.0,NaN,NaT,[7501206674994] 21531 Brocha uso general de 6'...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticket,NaN,NaN,2025-07-03 08:15:21,NaN,63016.0,NaN,Público en General,NaN,NaN,...,$ 119.00,NaN,$ 119.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
3,PZA,1.0,NaN,NaT,[7506240645290] 13882 Felpa para rodillo 9' x ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PZA,1.0,NaN,NaT,[7501206619223] 20323 Espátula flexible 5' con...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Explicación 1

- "Unnamed: \d+": Es un patrón de expresión regular que coincida con la cadena "Unnamed: " seguido de uno o más digitos (\d+). Este patrón suele ser usado en excels que tienen columnas sin nombre.

- regex=True: Especifica que el primer argumento es una expresión regular.

Anexos:
- https://stackoverflow.com/questions/2841550/what-does-d-mean-in-a-regular-expression

In [3200]:
ventas.columns = ventas.columns.str.replace("Unnamed: \d+", "", regex=True)

columnas = ventas.columns.to_list()
columnas

['Documento',
 '',
 '',
 'Fecha',
 '',
 'Folio',
 '',
 'Cliente',
 '',
 '',
 '',
 '',
 'Caja',
 '',
 '',
 '',
 '',
 'Usuario',
 '',
 '',
 '',
 '',
 '',
 'Folio F.',
 '',
 '',
 'Est',
 '',
 '',
 '',
 'Total   ',
 '',
 'Efectivo',
 'Cheque',
 'Crédito',
 'Transferencia',
 'Vales',
 'Tarjeta',
 'Anticipo',
 'SICAR Pagos']

## Explicación 2

Filtra las cadenas vacías de la lista de columnas. Revisar el archivo origen de SICAR

In [3201]:
columnas_con_nombre = [col for col in columnas if col != '']

In [3202]:
ventas = ventas[columnas_con_nombre]
ventas.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Folio F.,Est,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,NaN,V,$ 49.00,$ 49.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
1,PZA,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,NaN,V,$ 119.00,$ 119.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
3,PZA,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PZA,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


No hacemos un drop.na por la columna de "Folio F.", en donde la mayoría de los datos son nulos

In [3203]:
ventas = ventas.loc[(ventas["Documento"] == "Ticket") | (ventas["Documento"] == "Factura CFDI")]
ventas.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Folio F.,Est,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,NaN,V,$ 49.00,$ 49.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,NaN,V,$ 119.00,$ 119.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
7,Ticket,2025-07-03 08:19:15,63017.0,Público en General,Caja 1,admin,NaN,V,$ 12.00,$ 12.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00
9,Factura CFDI,2025-07-03 08:22:30,22736.0,VISION COMERCIAL DEPORTIVA,Caja 1,admin,22736.0,V,$ 279.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 279.00,$ 0.00,$ 0.00
12,Ticket,2025-07-03 08:26:35,63018.0,Público en General,Caja 1,admin,NaN,V,$ 326.00,$ 326.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00


## Explicación 3
Eliminamos todos los signos y comas de las columnas que contienen información numérica. Si no, no podemos hacer los cálculos necesarios.

Nota: La columna "Total" del archivo de ventas de SICAR tiene esos espacios en blanco.

In [3204]:
ventas['Total   '] = ventas["Total   "].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Efectivo'] = ventas["Efectivo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Cheque'] = ventas["Cheque"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Crédito'] = ventas["Crédito"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Transferencia'] = ventas["Transferencia"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Vales'] = ventas["Vales"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Tarjeta'] = ventas["Tarjeta"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['Anticipo'] = ventas["Anticipo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
ventas['SICAR Pagos'] = ventas["SICAR Pagos"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)


/tmp/ipython-input-3204-1681739492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ventas['Total   '] = ventas["Total   "].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
/tmp/ipython-input-3204-1681739492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ventas['Efectivo'] = ventas["Efectivo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
/tmp/ipython-input-3204-1681739492.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [3205]:
ventas.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Folio F.,Est,Total,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,NaN,V,49.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,NaN,V,119.0,119.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Ticket,2025-07-03 08:19:15,63017.0,Público en General,Caja 1,admin,NaN,V,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Factura CFDI,2025-07-03 08:22:30,22736.0,VISION COMERCIAL DEPORTIVA,Caja 1,admin,22736.0,V,279.0,0.0,0.0,0.0,0.0,0.0,279.0,0.0,0.0
12,Ticket,2025-07-03 08:26:35,63018.0,Público en General,Caja 1,admin,NaN,V,326.0,326.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sacamos los totales de todas las columnas de dinero

In [3206]:
total = ventas['Total   '].sum().round(2)
total

np.float64(64234.8)

In [3207]:
efectivo = ventas['Efectivo'].sum().round(2)
efectivo

np.float64(23590.2)

In [3208]:
cheque = ventas['Cheque'].sum().round(2)
cheque

np.float64(74.0)

In [3209]:
credito = ventas['Crédito'].sum().round(2)
credito

np.float64(0.0)

In [3210]:
transferencia = ventas['Transferencia'].sum().round(2)
transferencia

np.float64(8083.0)

In [3211]:
vales = ventas['Vales'].sum().round(2)
vales

np.float64(0.0)

In [3212]:
tarjeta = ventas['Tarjeta'].sum().round(2)
tarjeta

np.float64(32487.6)

In [3213]:
anticipo = ventas['Anticipo'].sum().round(2)
anticipo

np.float64(0.0)

In [3214]:
s_pagos = ventas['SICAR Pagos'].sum().round(2)
s_pagos

np.float64(0.0)

# Limpieza y transformación de SICAR Descuentos

In [3215]:
descuentos = pd.read_excel("Descuentos.xlsx", sheet_name=0, header=9)
descuentos.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,Documento,Unnamed: 1,Fecha,Unnamed: 3,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Unnamed: 23,Unnamed: 24,Descuento $\n,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Total
0,Ticket,NaN,2025-07-03 08:03:19,NaN,NaN,63015.0,NaN,Público en General,NaN,NaN,...,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,NaN,$ 49.00
1,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,[7501206674994] 21531 Brocha uso general de 6'...,...,$ 49.00,NaN,NaN,NaT,NaN,NaN,$ 49.00,NaN,NaN,NaN
2,Ticket,NaN,2025-07-03 08:15:21,NaN,NaN,63016.0,NaN,Público en General,NaN,NaN,...,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,NaN,$ 119.00
3,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,[7506240645290] 13882 Felpa para rodillo 9' x ...,...,$ 22.00,NaN,NaN,NaT,NaN,NaN,$ 22.00,NaN,NaN,NaN
4,NaN,NaN,NaT,NaN,NaN,NaN,PZA,NaN,1.0,[7501206619223] 20323 Espátula flexible 5' con...,...,$ 25.00,NaN,NaN,NaT,NaN,NaN,$ 25.00,NaN,NaN,NaN


Nota: Ver "Explicación 1"

In [3216]:
descuentos.columns = descuentos.columns.str.replace("Unnamed: \d+", "", regex=True)

columnas = descuentos.columns.to_list()
columnas

['Documento',
 '',
 'Fecha',
 '',
 '',
 'Folio',
 '',
 'Cliente',
 '',
 '',
 '',
 '',
 '',
 'Caja',
 '',
 '',
 'Usuario',
 '',
 '',
 'Est',
 '',
 'Desc %',
 '',
 '',
 '',
 'Descuento $\n',
 '',
 '',
 '',
 '',
 '',
 '',
 'Total']

Nota: Ver explicación 2

In [3217]:
columnas_con_nombre = [col for col in columnas if col != '']

columnas_con_nombre

['Documento',
 'Fecha',
 'Folio',
 'Cliente',
 'Caja',
 'Usuario',
 'Est',
 'Desc %',
 'Descuento $\n',
 'Total']

In [3218]:
descuentos = descuentos[columnas_con_nombre]
descuentos.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Est,Desc %,Descuento $\n,Total
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 49.00
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 119.00
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3219]:
descuentos = descuentos.loc[(descuentos["Documento"] == "Ticket") | (descuentos["Documento"] == "Factura CFDI")]
descuentos.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Est,Desc %,Descuento $\n,Total
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 49.00
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 119.00
7,Ticket,2025-07-03 08:19:15,63017.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 12.00
9,Factura CFDI,2025-07-03 08:22:30,22736.0,VISION COMERCIAL DEPORTIVA,Caja 1,admin,V,0.00%,$ 0.00,$ 279.00
12,Ticket,2025-07-03 08:26:35,63018.0,Público en General,Caja 1,admin,V,0.00%,$ 0.00,$ 326.00


Nota: Ver explicación 3

In [3220]:
descuentos['Total'] = descuentos["Total"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
descuentos['Descuento $\n'] = descuentos["Descuento $\n"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)

In [3221]:
descuentos.head()

,Documento,Fecha,Folio,Cliente,Caja,Usuario,Est,Desc %,Descuento $\n,Total
0,Ticket,2025-07-03 08:03:19,63015.0,Público en General,Caja 1,admin,V,0.00%,0.0,49.0
2,Ticket,2025-07-03 08:15:21,63016.0,Público en General,Caja 1,admin,V,0.00%,0.0,119.0
7,Ticket,2025-07-03 08:19:15,63017.0,Público en General,Caja 1,admin,V,0.00%,0.0,12.0
9,Factura CFDI,2025-07-03 08:22:30,22736.0,VISION COMERCIAL DEPORTIVA,Caja 1,admin,V,0.00%,0.0,279.0
12,Ticket,2025-07-03 08:26:35,63018.0,Público en General,Caja 1,admin,V,0.00%,0.0,326.0


In [3222]:
descuento = descuentos['Descuento $\n'].sum().round(2)
descuento

np.float64(107.4)

# Extracción ARQUEO

Consultar el archivo de "Cierre.xlsx"

In [3223]:
wb = load_workbook('Arqueo.xlsx', data_only=True)
ws = wb.worksheets[0]

In [3224]:
inicio_caja = ws['L10'].value
inicio_caja

800

In [3225]:
t_monedas_20 = 0
t_monedas_50 = 0
t_monedas1 = 0
t_monedas2 = 0
t_monedas5 = 0
t_monedas10 = 0

In [3226]:
try:
  t_monedas_20 += ws['E28'].value
except:
  t_monedas_20 += 0
try:
  t_monedas_20 += ws['E42'].value
except:
  t_monedas_20 += 0
try:
  t_monedas_20 += ws['E56'].value
except:
  t_monedas_20 += 0


In [3227]:
try:
  t_monedas_50 += ws['E29'].value
except:
  t_monedas_50 += 0
try:
  t_monedas_50 += ws['E43'].value
except:
  t_monedas_50 += 0
try:
  t_monedas_50 += ws['E57'].value
except:
  t_monedas_50 += 0


In [3228]:
t_monedas1 = 0
try:
  t_monedas1 += ws['E30'].value
except:
  t_monedas1 += 0
try:
  t_monedas1 += ws['E44'].value
except:
  t_monedas1 += 0
try:
  t_monedas1 += ws['E58'].value
except:
  t_monedas1 += 0


In [3229]:
try:
  t_monedas2 += ws['E31'].value
except:
  t_monedas2 += 0
try:
  t_monedas2 += ws['E45'].value
except:
  t_monedas2 += 0
try:
  t_monedas2 += ws['E59'].value
except:
  t_monedas2 += 0

try:
  t_monedas5 += ws['E32'].value
except:
  t_monedas5 += 0
try:
  t_monedas5 += ws['E46'].value
except:
  t_monedas5 += 0
try:
  t_monedas5 += ws['E60'].value
except:
  t_monedas5 += 0

try:
  t_monedas10 += ws['E33'].value
except:
  t_monedas10 += 0
try:
  t_monedas10 += ws['E47'].value
except:
  t_monedas10 += 0
try:
  t_monedas10 += ws['E61'].value
except:
  t_monedas10 += 0


In [3230]:
t_billetes20 = 0
t_billetes50 = 0
t_billetes100 = 0
t_billetes200 = 0
t_billetes500 = 0
t_billetes1000 = 0

In [3231]:
t_billetes20 = 0
try:
  t_billetes20 += ws['I28'].value
except:
  t_billetes20 += 0
try:
  t_billetes20 += ws['I42'].value
except:
  t_billetes20 += 0
try:
  t_billetes20 += ws['I56'].value
except:
  t_billetes20 += 0


In [3232]:
t_billetes50 = 0
try:
  t_billetes50 += ws['I29'].value
except:
  t_billetes50 += 0
try:
  t_billetes50 += ws['I43'].value
except:
  t_billetes50 += 0
try:
  t_billetes50 += ws['I57'].value
except:
  t_billetes50 += 0


t_billetes100 = 0
try:
  t_billetes100 += ws['I30'].value
except:
  t_billetes100 += 0
try:
  t_billetes100 += ws['I44'].value
except:
  t_billetes100 += 0
try:
  t_billetes100 += ws['I58'].value
except:
  t_billetes100 += 0


t_billetes200 = 0
try:
  t_billetes200 += ws['I31'].value
except:
  t_billetes200 += 0
try:
  t_billetes200 += ws['I45'].value
except:
  t_billetes200 += 0
try:
  t_billetes200 += ws['I59'].value
except:
  t_billetes200 += 0


t_billetes500 = 0
try:
  t_billetes500 += ws['I32'].value
except:
  t_billetes500 += 0
try:
  t_billetes500 += ws['I46'].value
except:
  t_billetes500 += 0
try:
  t_billetes500 += ws['I60'].value
except:
  t_billetes500 += 0


t_billetes1000 = 0
try:
  t_billetes1000 += ws['I33'].value
except:
  t_billetes1000 += 0
try:
  t_billetes1000 += ws['I47'].value
except:
  t_billetes1000 += 0
try:
  t_billetes1000 += ws['I61'].value
except:
  t_billetes1000 += 0


In [3233]:
t_m_20 = t_monedas_20 * 0.20
t_m_50 = t_monedas_50 * 0.50
t_m1 = t_monedas1
t_m2 = t_monedas2 * 2
t_m5 = t_monedas5 * 5
t_m10 = t_monedas10 * 10

In [3234]:
t_b20 = t_billetes20 * 20
t_b50 = t_billetes50 * 50
t_b100 = t_billetes100 * 100
t_b200 = t_billetes200 * 200
t_b500 = t_billetes500 * 500
t_b1000 = t_billetes1000 * 1000

## Arqueo de efectivo

In [3235]:
total_monedas = t_m_20 + t_m_50 + t_m1 + t_m2 + t_m5 + t_m10
total_billetes = t_b20 + t_b50 + t_b100 + t_b200 + t_b500 + t_b1000
total_caja = total_monedas + total_billetes

In [3236]:
try:
  getnet = ws["K120"].value
except:
  getnet = 0
try:
  evo = ws["H120"].value
except:
  evo = 0
try:
  a_express = ws["E120"].value
except:
  a_express = 0
try:
  m_pago = ws["N120"].value
except:
  m_pago = 0

In [3237]:
total_terminales = (getnet if getnet is not None else 0) + (evo if evo is not None else 0) + (a_express if a_express is not None else 0) + (m_pago if m_pago is not None else 0)
total_terminales

32476.5

In [3238]:
caja_inicial = ws["L10"].value
salidas_en_efectivo = 0
caja_abono = 0
venta_abono = 0
salidas_abono = ws['L89'].value

In [3239]:
totales_cargo = efectivo + caja_inicial + salidas_en_efectivo
totales_abono = caja_abono + venta_abono + salidas_abono

In [3240]:
caja_de_cambio = ws["L140"].value

In [3241]:
saldo = totales_cargo - totales_abono
saldo_efectivo = total_caja - saldo

Nota: Obtener estos datos del archivo correspondiente. *pendiente

In [3242]:
apartados_finalizados = 0

In [3243]:
adelantos_iniciados = 0

In [3244]:
devoluciones = 0

## Saldo efectivo final

In [3245]:
saldo_efectivo_con_pendientes = saldo_efectivo - apartados_finalizados

In [3246]:
saldo_efectivo_con_adelantos = saldo_efectivo_con_pendientes - adelantos_iniciados

In [3247]:
saldo_efectivo_final = saldo_efectivo_con_adelantos + devoluciones

In [3248]:
saldo_tarjeta = total_terminales - tarjeta

In [3249]:
adelantos_tarjeta = 0

In [3250]:
saldo_final_tarjeta = saldo_tarjeta - adelantos_tarjeta

In [3251]:
aFinalizaods_tarjeta = 0

In [3252]:
final_tarjeta = saldo_final_tarjeta + aFinalizaods_tarjeta

In [3253]:
suma_tienda = credito + tarjeta + efectivo + vales + transferencia
ingreso_total_tienda = suma_tienda - 0
'''Al ingreso total de la tienda se le resta un valor el cual actualmente desconozco, se queda el 0 como placeholder'''

'Al ingreso total de la tienda se le resta un valor el cual actualmente desconozco, se queda el 0 como placeholder'

Nota: Obtener los valores de marketplace cuando se conozca su origen

In [3254]:
total_marketplace = 0

In [3255]:
total_global = total_marketplace + ingreso_total_tienda

In [3256]:
total_final = total - total_global

# Creacion del excel de Reporte Cierre Diario (RCD)

In [3257]:
cierre = Workbook()
cs = cierre.active
cs.title = "Cierre"

In [3258]:
cs['B2'] = "Reporte Cierre Diario (RCD)"
cs.merge_cells(start_row=2, start_column=2, end_row=2, end_column=9)
cs['E4'] = "CONCEPTO"
cs['F4'] = "CARGO"
cs['G4'] = "ABONO"

In [3259]:
cs['E5'] = "CAJA INICIAL EFECTIVO"
cs['E6'] = "VENTA EFECTIVO SICAR"
cs['E7'] = "SALIDAS EN EFECTIVO"
cs['E8'] = "TOTALES"

In [3260]:
cs['F5'] = inicio_caja
cs['F6'] = efectivo
cs['F7'] = 0
cs['F8'] = totales_cargo

In [3261]:
cs['G5'] = 0
cs['G6'] = 0
cs['G7'] = salidas_abono
cs['G8'] = totales_abono

In [3262]:
cs['F11'] = "CAJA DE CAMBIO"
cs['G11'] = caja_de_cambio

In [3263]:
cs['F13'] = "SALDO"
cs['F14'] = "ARQUEO"
cs['F15'] = "SALDO EFECTIVO"
cs['F16'] = "APARTADOS FINALIZADOS"
cs['F17'] = "SALDO EFECTIVO CON PENDIENTES"
cs['F18'] = "ADELANTOS INICIADOS"
cs['F19'] = "SALDO EFECTIVO CON ADELANTOS"
cs['F20'] = "DEVOLUCIONES"
cs['F21'] = "SALDO EFECTIVO FINAL"

In [3264]:
cs['G13'] = saldo
cs['G14'] = total_caja
cs['G15'] = saldo_efectivo
cs['G16'] = apartados_finalizados
cs['G17'] = saldo_efectivo_con_pendientes
cs['G18'] = adelantos_iniciados
cs['G19'] = saldo_efectivo_con_adelantos
cs['G20'] = devoluciones
cs['G21'] = saldo_efectivo_final

In [3265]:
cs['F23'] = "PAGO CON TARJETA (SICAR)"
cs['F24'] = "ARQUEO TARJETA (TERMINALES)"
cs['F25'] = "SALDO TARJETA"
cs['F26'] = "ADELANTOS INICIADOS CON TARJETA"
cs['F27'] = "SALDO FINAL TARJETA"
cs['F28'] = "ADELANTOS FINALIZADOS CON TARJETA"
cs['F29'] = "SALDO FINAL TARJETA"

In [3266]:
cs['G23'] = tarjeta
cs['G24'] = total_terminales
cs['G25'] = saldo_tarjeta
cs['G26'] = adelantos_tarjeta
cs['G27'] = final_tarjeta
cs['G28'] = adelantos_tarjeta
cs['G29'] = saldo_final_tarjeta

In [3267]:
cs['F31'] = "TRANSFERENCIAS CLIENTES"
cs['F32'] = "TRANSFERENCIAS APARTADO"

In [3268]:
cs['G31'] = transferencia
cs['G32'] = 0

In [3269]:
cs['F34'] = "CRÉDITO"
cs['G34'] = credito

In [3270]:
cs['F36'] = "INGRESO TOTAL TIENDA"
cs['F37'] = "CREDITO"
cs['F38'] = "TARJETA"
cs['F39'] = "EFECTIVO"
cs['F40'] = "VALES FERRENACIONAL"
cs['F41'] = "TRANSFERENCIA"
cs['F42'] = "CHEQUE"
cs['F43'] = "MARKETPLACES"
cs['F44'] = "AMAZON"
cs['F45'] = "LIVERPOOL"
cs['F46'] = "COPPEL"
cs['F47'] = "WALMART"
cs['F48'] = "LINIO"
cs['F49'] = "CLARO-SHOP"
cs['F50'] = "SEARS"
cs['F51'] = "FAMSA"
cs['F52'] = "CYBER"
cs['F53'] = "MARKETPLACES"

cs['F55'] = "TOTAL GLOBAL"

In [3271]:
cs['G36'] = ingreso_total_tienda
cs['G37'] = credito
cs['G38'] = tarjeta
cs['G39'] = efectivo
cs['G40'] = vales
cs['G41'] = transferencia
cs['G42'] = cheque

cs['G44'] = 0
cs['G45'] = 0
cs['G46'] = 0
cs['G47'] = 0
cs['G48'] = 0
cs['G49'] = 0
cs['G50'] = 0
cs['G51'] = 0
cs['G52'] = 0
cs['G53'] = total_marketplace

cs['G55'] = total_global
cs['H55'] = total
cs['I55'] = total_final

In [3272]:
archivo = "RCD " + str(datetime.datetime.strftime(datetime.datetime.now(), "%d-%m-%Y")) + ".xlsx"

## Formato
Se le empieza a dar formato al archivo Excel final

### Texto
Se crean dos plantillas de texto
- font es una fuente Comfortaa de tamaño 12 y color negro
- font_w es similar, pero de color blanco

In [3273]:
font = Font(name="Comfortaa", size=12, bold=False, italic=False, color="000000")
font_w = Font(name="Comfortaa", size=12, bold=False, italic=False, color="ffffff")

In [3274]:
cs.column_dimensions['E'].width = 30
cs.column_dimensions['F'].width = 55
cs.column_dimensions['G'].width = 15
cs.column_dimensions['H'].width = 15
cs.column_dimensions['I'].width = 15
cs.column_dimensions['J'].width = 15

In [3275]:
for row in range(1, 56):
  for col in range(1, 10):
    cs.cell(row=row, column=col).font = font

### Color de fondo
Se crean los colores de fondo
- fill_y -> amarillo
- fill_b -> negro
- fill_gy -> gris
- fill_dg -> verde oscuro
- fill_lg -> verde claro
- fill_r -> rojo

In [3276]:
fill_y = PatternFill(fill_type="solid", start_color="ffeb9c", end_color="ffeb9c")
fill_b = PatternFill(fill_type="solid", start_color="000000", end_color="000000")
fill_gy = PatternFill(fill_type="solid", start_color="e7e6e6", end_color="e7e6e6")
fill_dg = PatternFill(fill_type="solid", start_color="385623", end_color="385623")
fill_lg = PatternFill(fill_type="solid", start_color="c6efce", end_color="c6efce")
fill_r = PatternFill(fill_type="solid", start_color="ffc7ce", end_color="ffc7ce")

In [3277]:
cs.cell(row=5, column=6).fill = fill_y
cs.cell(row=14, column=7).fill = fill_y
cs.cell(row=24, column=7).fill = fill_y

cs.cell(2, 2).fill = fill_b
cs.cell(2, 2).font = font_w
cs.cell(4, 5).fill = fill_b
cs.cell(4, 5).font = font_w
cs.cell(4, 6).fill = fill_b
cs.cell(4, 6).font = font_w
cs.cell(4, 7).fill = fill_b
cs.cell(4, 7).font = font_w

cs.cell(8, 5).fill = fill_b
cs.cell(8, 5).font = font_w
cs.cell(8, 6).fill = fill_b
cs.cell(8, 6).font = font_w
cs.cell(8, 7).fill = fill_b
cs.cell(8, 7).font = font_w

In [3278]:
cs.cell(53, 6).fill = fill_b
cs.cell(53, 6).font = font_w
cs.cell(53, 7).fill = fill_b
cs.cell(53, 7).font = font_w

### Formato de número
Crea el formato de moneda, con el signo $ ubicado a la izquierda, separando los miles con comas y los decimales con punto

In [3279]:
num_form = '$#,##0.00'

In [3280]:
for row in range(1, 56):
  for col in range(1, 10):
    cs.cell(row, col).number_format = num_form
    cs.cell(row, col).alignment = Alignment(horizontal="right")

cs.cell(2, 2).alignment = Alignment(horizontal="center")

In [3281]:
cs.cell(55, 6).fill = fill_b
cs.cell(55, 6).font = font_w

cs.cell(55, 7).fill = fill_lg
cs.cell(55, 8).fill = fill_lg

cs.cell(55, 9).fill = fill_r

cs.cell(43, 6).fill = fill_b
cs.cell(43, 6).font = font_w
cs.cell(43, 7).fill = fill_b
cs.cell(43, 7).font = font_w

cs.cell(36, 6).fill = fill_dg
cs.cell(36, 6).font = font_w
cs.cell(36, 7).fill = fill_lg

cs.cell(34, 6).fill = fill_dg
cs.cell(34, 6).font = font_w
cs.cell(34, 7).fill = fill_lg

cs.cell(32, 6).fill = fill_dg
cs.cell(32, 6).font = font_w
cs.cell(32, 7).fill = fill_lg

cs.cell(31, 6).fill = fill_dg
cs.cell(31, 6).font = font_w
cs.cell(31, 7).fill = fill_lg

cs.cell(29, 6).fill = fill_dg
cs.cell(29, 6).font = font_w
cs.cell(29, 7).fill = fill_lg

cs.cell(42, 6).fill = fill_dg
cs.cell(42, 6).font = font_w
cs.cell(42, 7).fill = fill_lg

cs.cell(21, 6).fill = fill_dg
cs.cell(21, 6).font = font_w
cs.cell(21, 7).fill = fill_lg

cs.cell(6, 6).fill = fill_lg

In [3282]:
for row in range(13, 21):
  for col in range(6, 8):
    cs.cell(row, col).fill = fill_gy

for row in range(23, 29):
  for col in range(6, 8):
    cs.cell(row, col).fill = fill_gy

cs.cell(11, 6).fill = fill_gy
cs.cell(11, 7).fill = fill_gy

cs.cell(7, 7).fill = fill_r

In [3283]:
for row in range(37, 42):
  for col in range(6, 8):
    cs.cell(row, col).fill = fill_gy

for row in range(44, 53):
  for col in range(6, 8):
    cs.cell(row, col).fill = fill_gy

In [3284]:
cierre.save(archivo)

## Adición de la hoja de Arqueo efectivo

In [3285]:
a_efectivo = load_workbook(archivo)
a_efectivo.create_sheet("Arqueo de efectivo")
a_s = a_efectivo.worksheets[1]

In [3286]:
a_s["C3"] = "ARQUEO DE EFECTIVO"
a_s["D15"] = "MONEDAS"
a_s["D16"] = "VALOR"
a_s["E16"] = "CANTIDAD"
a_s["F16"] = "TOTAL"

In [3287]:
cs.merge_cells(start_row=15, start_column=4, end_row=15, end_column=6)
cs.merge_cells(start_row=15, start_column=8, end_row=15, end_column=10)

In [3288]:
a_s["D17"] = "0.20"
a_s["D18"] = "0.50"
a_s["D19"] = "1.00"
a_s["D20"] = "2.00"
a_s["D21"] = "5.00"
a_s["D22"] = "10.00"
a_s["D23"] = "Total Monedas"

In [3289]:
a_s["E17"] = t_monedas_20
a_s["E18"] = t_monedas_50
a_s["E19"] = t_monedas1
a_s["E20"] = t_monedas2
a_s["E21"] = t_monedas5
a_s["E22"] = t_monedas10

In [3290]:
a_s["F17"] = t_m_20
a_s["F18"] = t_m_50
a_s["F19"] = t_m1
a_s["F20"] = t_m2
a_s["F21"] = t_m5
a_s["F22"] = t_m10
a_s["F23"] = total_monedas

In [3291]:
a_s["H15"] = "BILLETES"
a_s["H16"] = "VALOR"
a_s["I16"] = "CANTIDAD"
a_s["J16"] = "TOTAL"

In [3292]:
a_s["H17"] = "20.00"
a_s["H18"] = "50.00"
a_s["H19"] = "100.00"
a_s["H20"] = "200.00"
a_s["H21"] = "500.00"
a_s["H22"] = "1000.00"
a_s["H23"] = "Total Billetes"

In [3293]:
a_s["I17"] = t_billetes20
a_s["I18"] = t_billetes50
a_s["I19"] = t_billetes100
a_s["I20"] = t_billetes200
a_s["I21"] = t_billetes500
a_s["I22"] = t_billetes1000

In [3294]:
a_s["J17"] = t_b20
a_s["J18"] = t_b50
a_s["J19"] = t_b100
a_s["J20"] = t_b200
a_s["J21"] = t_b500
a_s["J22"] = t_b1000
a_s["J23"] = total_billetes

In [3295]:
a_s["L12"] = "TOTAL EN CAJA"
a_s["L13"] = total_caja

### Formato

In [3296]:
a_s.merge_cells(start_row=15, start_column=4, end_row=15, end_column=6)

In [3297]:
a_s.merge_cells(start_row=15, start_column=8, end_row=15, end_column=10)

In [3298]:
a_s.merge_cells(start_row=23, start_column=4, end_row=23, end_column=5)
a_s.merge_cells(start_row=23, start_column=8, end_row=23, end_column=9)
a_s.merge_cells(start_row=12, start_column=12, end_row=12, end_column=14)
a_s.merge_cells(start_row=3, start_column=3, end_row=3, end_column=13)

In [3299]:
for row in range(1, 56):
  for col in range(1, 15):
    a_s.cell(row=row, column=col).font = font

for row in range(1, 24):
  for col in range(1, 11):
    a_s.cell(row=row, column=col).alignment = Alignment(horizontal="right")

a_s.cell(3, 3).alignment = Alignment(horizontal="center")
a_s.cell(3, 3).fill = fill_b
a_s.cell(3, 3).font = font_w

a_s.cell(15, 4).alignment = Alignment(horizontal="center")
a_s.cell(15, 4).fill = fill_b
a_s.cell(15, 4).font = font_w

a_s.cell(15, 8).alignment = Alignment(horizontal="center")
a_s.cell(15, 8).fill = fill_b
a_s.cell(15, 8).font = font_w

for row in range(17, 23):
  a_s.cell(row, 4).number_format = num_form
  a_s.cell(row, 8).number_format = num_form
  a_s.cell(row, 6).number_format = num_form
  a_s.cell(row, 10).number_format = num_form

a_s.cell(13, 12).number_format = num_form
a_s.cell(23, 6).number_format = num_form
a_s.cell(23, 10).number_format = num_form

In [3300]:
for row in range(16, 23):
  for col in range(4, 7):
    a_s.cell(row, col).fill = fill_gy

for row in range(16, 23):
  for col in range(8, 11):
    a_s.cell(row, col).fill = fill_gy

a_s.cell(23, 4).fill = fill_dg
a_s.cell(23, 4).font = font_w
a_s.cell(23, 8).fill = fill_dg
a_s.cell(23, 8).font = font_w

a_s.cell(23, 6).fill = fill_lg
a_s.cell(23, 10).fill = fill_lg

a_s.cell(12, 12).fill = fill_b
a_s.cell(12, 12).font = font_w

In [3301]:
a_s.column_dimensions['D'].width = 20
a_s.column_dimensions['H'].width = 20

a_s.column_dimensions['E'].width = 25
a_s.column_dimensions['I'].width = 25

a_s.column_dimensions['F'].width = 20
a_s.column_dimensions['J'].width = 20

a_s.column_dimensions['L'].width = 20

In [3302]:
a_efectivo.save(archivo)

## Apartados
El tema de los apartados todavía no está resuelto. Esto se quedará como un placeholder. Mover cuando se tenga claro ese tema.


In [3303]:
'''
Trae excel de apartados iniciados (APA INI)
apa_ini = load_workbook('APA INI.xlsx', data_only=True)

Trae excel de apartados finalizados (APA FIN)
apa_fin = load_workbook('APA FIN.xlsx', data_only=True)

Traer excel de devoluciones y cancelaciones (?) (DeCa)
de_ca = load_workbook('DeCa.xlsx', data_only=True)

En la sección del reporte de cierre:
apartados_finalizados = apa_fin["M13"]
adelantos_iniciados = apa_ini["L7"]
devoluciones = de_ca["M13"]
'''

'\nTrae excel de apartados iniciados (APA INI)\napa_ini = load_workbook(\'APA INI.xlsx\', data_only=True)\n\nTrae excel de apartados finalizados (APA FIN)\napa_fin = load_workbook(\'APA FIN.xlsx\', data_only=True)\n\nTraer excel de devoluciones y cancelaciones (?) (DeCa)\nde_ca = load_workbook(\'DeCa.xlsx\', data_only=True)\n\nEn la sección del reporte de cierre:\napartados_finalizados = apa_fin["M13"]\nadelantos_iniciados = apa_ini["L7"]\ndevoluciones = de_ca["M13"]\n'

## Arqueo de terminales

In [3304]:
a_term = load_workbook(archivo)
a_term.create_sheet("Arqueo de terminales")
a_t = a_term.worksheets[2]

In [3305]:
a_t['C3'] = "ARQUEO DE CAJA DIARIO FERE PRO"
a_t.merge_cells(start_row=3, start_column=3, end_row=3, end_column=12)

In [3306]:
a_t['C5'] = "FECHA:"
a_t['C6'] = "HORA INICO:"
a_t['C7'] = "HORA TÉRMINO:"
a_t['I5'] = "ARQUEO N°:"

a_t['D10'] = "SALDO INICIAL:"

a_t['D13'] = "TARJETA"
a_t['E13'] = "GETNET SAN"
a_t['F13'] = "BBVA"
a_t['G13'] = "EVO"
a_t['H13'] = "AMEX"
a_t['K13'] = "TOTAL"
a_t['L13'] = total_terminales

a_t['D14'] = "TOTAL POR TERMINAL"
a_t['E14'] = getnet
a_t['F14'] = a_express
a_t['G14'] = evo
a_t['H14'] = a_express

### Formato

In [3307]:
for row in range(1, 25):
  for col in range(1, 13):
    a_t.cell(row=row, column=col).font = font

In [3308]:
a_t.column_dimensions['D'].width = 30
a_t.column_dimensions['E'].width = 25
a_t.column_dimensions['F'].width = 25
a_t.column_dimensions['G'].width = 25
a_t.column_dimensions['H'].width = 25
a_t.column_dimensions['L'].width = 25

In [3309]:
fill_lb = PatternFill(fill_type="solid", start_color="00b0f0", end_color="00b0f0")
fill_db = PatternFill(fill_type="solid", start_color="0040f0", end_color="0040f0")
fill_dr = PatternFill(fill_type="solid", start_color="c00000", end_color="c00000")

In [3310]:
a_t.cell(3, 3).alignment = Alignment(horizontal="center")
a_t.cell(3, 3).fill = fill_b
a_t.cell(3, 3).font = font_w

In [3311]:
a_t.cell(13, 5).fill = fill_dr
a_t.cell(13, 5).font = font_w
a_t.cell(13, 6).fill = fill_db
a_t.cell(13, 6).font = font_w
a_t.cell(13, 7).fill = fill_lb
a_t.cell(13, 8).fill = fill_b
a_t.cell(13, 8).font = font_w

In [3312]:
a_t.cell(14, 5).fill = fill_dr
a_t.cell(14, 5).font = font_w
a_t.cell(14, 6).fill = fill_db
a_t.cell(14, 6).font = font_w
a_t.cell(14, 7).fill = fill_lb
a_t.cell(14, 8).fill = fill_b
a_t.cell(14, 8).font = font_w

In [3313]:
for row in range(14, 24):
  for col in range(5, 9):
    a_t.cell(row, col).number_format = num_form

a_t.cell(13, 12).number_format = num_form

In [3314]:
for row in range(8, 24):
  for col in range(4, 13):
    a_t.cell(row, col).alignment = Alignment(horizontal="right")

In [3315]:
a_term.save(archivo)